<a href="https://colab.research.google.com/github/20v100/Coursera_Capstone/blob/master/JOC_Coursera_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[INTRODUCTION](#scrollTo=xbo8RHP8sIjZ)

>[DATA OVERVIEW](#scrollTo=bUxnorBl9l8a)

>[METHODOLOGY](#scrollTo=uGVCbx0--GFr)

>>[Libraries, Packages and Credentials](#scrollTo=mXWrDGUZAuOm)

>[RESULTS](#scrollTo=7euvspf9-GVG)

>>[Data Loading](#scrollTo=yX5VtnyOC6Xk)

>>>[Defining cities to analyse](#scrollTo=02kG1lnQD80Q)

>>>[Loading Foursquare information on the selected cities](#scrollTo=5sbYmQ9IEXII)

>>[Data Wrangling](#scrollTo=P9thZXYBDKIg)

>>[Onehot encoding](#scrollTo=Ugf0BUaClnXR)

>>[K-Means Analysis](#scrollTo=hJI0WvBLZzVh)

>>>[Model fitting](#scrollTo=56DpSjQ3a1P4)

>>>[Mapping the clusters](#scrollTo=47aOcSZ8eYAu)

>[DISCUSSION](#scrollTo=sxBellKm-Ggd)

>[CONCLUSION](#scrollTo=pva0qJ5g71hT)



#1. INTRODUCTION
*15 mark*

Introduction where you discuss the business problem and who would be interested in this project.

>sjldf;dsfj

>sldjfskldfjskdfj


#2. DATA OVERVIEW
*15 marks*

Data where you describe the data that will be used to solve the problem and the source of the data.



#3. METHODOLOGY
Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, and what machine learnings were used and why.

##Libraries, Packages and Credentials

In [0]:
# import the libraries

# Basic Libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
print('All imported')
import seaborn as sns

# plotting library
!pip install folium
import folium

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

All imported
Libraries imported.


In [0]:
#Four Square Credentials
CLIENT_ID = 'Z4G102XHUFAUSKXKLFW01UEVDEO3XIDLWNTGEWAKJDNN21JW' # your Foursquare ID
CLIENT_SECRET = 'TFQ4CBEKWNASZD4FROH4NHUS5PWSGRT2A14RWA3UFRFC54EU' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z4G102XHUFAUSKXKLFW01UEVDEO3XIDLWNTGEWAKJDNN21JW
CLIENT_SECRET:TFQ4CBEKWNASZD4FROH4NHUS5PWSGRT2A14RWA3UFRFC54EU


#4. RESULTS
Results section where you discuss the results

##Data Loading

###Defining cities to analyse 

In [0]:
cities = {'name': ['Montreal', 'Toronto', 'Calgary', 'Vancouver', 'New York', 'Denver', 'Dallas', 'Miami', 'Houton', 'Washington', 'Los Angeles'],
         'lat': [45.5017, 43.6532, 49.2827, 51.0486, 40.7128, 39.7392, 32.7767, 25.7617, 29.7604, 38.9072, 34.0522], 
         'long': [-73.5673, -79.3832, -123.1207, -114.0708, -74.0060, -104.9903, -96.7970, -80.1918, -95.3698, -77.0369, 118.2437]
        }

city_df = pd.DataFrame(cities)
city_df.head()

,lat,long,name
0,45.5017,-73.5673,Montreal
1,43.6532,-79.3832,Toronto
2,49.2827,-123.1207,Calgary
3,51.0486,-114.0708,Vancouver
4,40.7128,-74.0060,New York


In [0]:
# create map

latitude  = 40
longitude = -105.2551
zoom_start = 4

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=zoom_start)

# # set color scheme for the clusters
# x = np.arange(city_df['name'])
# ys = [i+x+(i*x)**2 for i in range(city_df['name'])]
# colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
# rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_df['lat'], city_df['long'], city_df['name'], city_df['name']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
#         color=rainbow[cluster-1],
        fill=True,
#         fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

###Loading Foursquare information on the selected cities

In [0]:
# Function that explore cities
# Radius is 2 KM
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['city', 
                  'city Latitude', 
                  'city Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [0]:
# Resquest info on venues from Foursquare
city_venues = getNearbyVenues(names=city_df['name'],
                                   latitudes=city_df['lat'],
                                   longitudes=city_df['long']
                                  )

Montreal
Toronto
Calgary
Vancouver
New York
Denver
Dallas
Miami
Houton
Washington
Los Angeles


In [0]:
city_venues.head()

,city,city Latitude,city Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Montreal,45.5017,-73.5673,The Keg Steakhouse & Bar,45.500730,-73.568971,Steakhouse
1,Montreal,45.5017,-73.5673,"96,9 CKOI",45.500000,-73.565269,Music Venue
2,Montreal,45.5017,-73.5673,Escondite,45.503045,-73.567888,Taco Place
3,Montreal,45.5017,-73.5673,Tunnel Espresso,45.502167,-73.569954,Coffee Shop
4,Montreal,45.5017,-73.5673,Il Focolaio,45.504009,-73.568213,Pizza Place


In [0]:
city_venues.shape

(1000, 7)

In [0]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 105 uniques categories.


In [0]:
city_venues.groupby(['city']).size()

city
Calgary       100
Dallas        100
Denver        100
Houton        100
Miami         100
Montreal      100
New York      100
Toronto       100
Vancouver     100
Washington    100
dtype: int64

##Data Wrangling

##Onehot encoding

In [0]:
# one hot encoding
venues_onehot = pd.get_dummies(city_venues[['Venue Category']], prefix="", prefix_sep="")

# venues_onehot.to_csv('venues_onehot.csv')
venues_onehot.head()

,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
venues_onehot.columns

Index(['American Restaurant', 'Antique Shop', 'Aquarium', 'Arcade',
       'Argentinian Restaurant', 'Art Gallery', 'Art Museum',
       'Arts & Crafts Store', 'Asian Restaurant', 'Athletics & Sports',
       ...
       'Trail', 'Turkish Restaurant', 'Vegetarian / Vegan Restaurant',
       'Vietnamese Restaurant', 'Whisky Bar', 'Wine Bar', 'Wine Shop',
       'Wings Joint', 'Women's Store', 'Yoga Studio'],
      dtype='object', length=195)

In [0]:
venues_onehot.shape

(1000, 195)

In [0]:
# Merging the onehot with the city information
city_onehot = city_venues.join(venues_onehot)
city_onehot.drop(['city Latitude', 'city Longitude','Venue Latitude', 'Venue Longitude'] ,1, inplace=True)
# city_onehot.to_csv('city _onehot.csv')
city_onehot.shape

(1000, 198)

In [0]:
city_onehot.head()

,city,Venue,Venue Category,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Montreal,The Keg Steakhouse & Bar,Steakhouse,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Montreal,"96,9 CKOI",Music Venue,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Montreal,Escondite,Taco Place,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Montreal,Tunnel Espresso,Coffee Shop,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Montreal,Il Focolaio,Pizza Place,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
# Next, let's group rows by cities and by taking the mean of the frequency of occurrence of each category
city_grouped = city_onehot.groupby(['city']).mean()
city_grouped.reset_index(inplace=True)
city_grouped.head(10)

,city,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Calgary,0.02,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01
1,Dallas,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00
2,Denver,0.03,0.00,0.00,0.00,0.00,0.01,0.02,0.00,0.00,...,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.03
3,Houton,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,...,0.02,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.00
4,Miami,0.02,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.01,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01
5,Montreal,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,...,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01
6,New York,0.02,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.01,...,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.02
7,Toronto,0.05,0.00,0.00,0.00,0.00,0.00,0.01,0.01,0.02,...,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00
8,Vancouver,0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.01,0.02,0.00,0.01,0.00,0.00,0.00,0.02
9,Washington,0.05,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,...,0.00,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.03


In [0]:
city_grouped.shape

(10, 197)

In [0]:
city_grouped.to_csv('city_grouped.csv')

##K-Means Analysis

###Model **fitting**

In [0]:
# set number of clusters
kclusters = 3

city_grouped_clustering = city_grouped.drop(['city'], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 1, 0, 2, 1, 2, 1, 1], dtype=int32)

In [0]:
# Merge back the labels

city_clusters = city_grouped

# add clustering labels
city_clusters['Cluster Labels'] = kmeans.labels_

# add lat long
city_clusters = pd.merge(city_df, city_clusters, left_on='name', right_on='city')
city_clusters.drop('name', 1, inplace=True)

city_clusters.head()

,lat,long,city,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster Labels
0,45.5017,-73.5673,Montreal,0.00,0.00,0.0,0.0,0.0,0.00,0.00,...,0.0,0.01,0.00,0.0,0.00,0.00,0.0,0.00,0.01,2
1,43.6532,-79.3832,Toronto,0.05,0.00,0.0,0.0,0.0,0.00,0.01,...,0.0,0.01,0.00,0.0,0.00,0.00,0.0,0.01,0.00,2
2,49.2827,-123.1207,Calgary,0.02,0.00,0.0,0.0,0.0,0.01,0.00,...,0.0,0.00,0.00,0.0,0.01,0.00,0.0,0.00,0.01,0
3,51.0486,-114.0708,Vancouver,0.03,0.00,0.0,0.0,0.0,0.00,0.00,...,0.0,0.01,0.02,0.0,0.01,0.00,0.0,0.00,0.02,1
4,40.7128,-74.0060,New York,0.02,0.01,0.0,0.0,0.0,0.00,0.00,...,0.0,0.01,0.00,0.0,0.00,0.01,0.0,0.01,0.02,1


###Mapping the clusters

In [0]:
# create map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(city_clusters['lat'], city_clusters['long'], city_clusters['city'], city_clusters['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#5. DISCUSSION
Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

In [0]:
city_clusters.loc[city_clusters['Cluster Labels'] == 0, city_clusters.columns[[1] + list(range(5, city_clusters.shape[1]))]]

,long,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster Labels
2,-123.1207,0.0,0.0,0.00,0.01,0.0,0.0,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0,0.01,0
7,-80.1918,0.0,0.0,0.03,0.00,0.0,0.0,0.01,0.01,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.01,0


In [0]:
# city_clusters.loc[city_clusters['Cluster Labels'] == 2, city_clusters.columns[[1] + list(range(5, city_clusters.shape[1]))]].columns[1]
# for c in city_clusters.loc[city_clusters['Cluster Labels'] == 2, city_clusters.columns[[1] + list(range(5, city_clusters.shape[1]))]]:
#   print(c)
#   type(c)
# [print(c, c) for c in city_clusters if city_clusters.loc(:,'Aquarium':'Yoga Studio')]
# city_clusters_heatmap = city_clusters.reindex(city_clusters['city'])

# index= list(city_clusters.sort_values('Cluster Labels', ascending=False)['city'])
# index
# city_clusters
# city_clusters_heatmap = pd.DataFrame(index=index)
# city_clusters_heatmap
# city_clusters_heatmap = city_clusters.drop(['lat', 'long'], axis=1)
# city_clusters_heatmap2 = pd.merge(city_clusters_heatmap, city_clusters, left_on=index, right_on='city')
# city_clusters_heatmap2

# city_clusters.reset_index('city', inplace=True)
city_clusters_t = city_clusters.sort_values('Cluster Labels', ascending=False)
city_clusters_t  = city_clusters_t.transpose()
city_clusters_t2 = city_clusters_t.loc['American Restaurant':'Yoga Studio', :]

city_clusters_t2

# from matplotlib import pyplot

# dims = (12, 50)
# fig, ax = pyplot.subplots(figsize=dims)
# sns.heatmap(ax=ax, data=city_clusters_t2)
sns.heatmap(city_clusters_t2)

# plt.show()

TypeError: ignored

In [0]:
city_clusters.loc[city_clusters['Cluster Labels'] == 1, city_clusters.columns[[1] + list(range(5, city_clusters.shape[1]))]]

,long,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster Labels
3,-114.0708,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,...,0.00,0.01,0.02,0.00,0.01,0.00,0.00,0.00,0.02,1
4,-74.0060,0.00,0.00,0.0,0.00,0.00,0.0,0.01,0.0,0.01,...,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.02,1
5,-104.9903,0.00,0.00,0.0,0.01,0.02,0.0,0.00,0.0,0.00,...,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.03,1
6,-96.7970,0.00,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,1
8,-95.3698,0.01,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,...,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.00,0.00,1
9,-77.0369,0.00,0.01,0.0,0.00,0.00,0.0,0.00,0.0,0.00,...,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.03,1


In [0]:
city_clusters.loc[city_clusters['Cluster Labels'] == 2, city_clusters.columns[[1] + list(range(5, city_clusters.shape[1]))]]

,long,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Cluster Labels
0,-73.5673,0.0,0.0,0.0,0.0,0.00,0.01,0.00,0.0,0.0,...,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.00,0.01,2
1,-79.3832,0.0,0.0,0.0,0.0,0.01,0.01,0.02,0.0,0.0,...,0.0,0.01,0.0,0.0,0.0,0.0,0.0,0.01,0.00,2


#6. CONCLUSION
Conclusion section where you conclude the report.